Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 736 µs, sys: 4.12 ms, total: 4.86 ms
Wall time: 10.8 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.255.71/10216/1 Dashboard: http://10.20.255.71:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 304 ms, sys: 44.3 ms, total: 349 ms
Wall time: 2.71 s


[1.7219118881892057,
 1.0376439415761096,
 1.2745297433453466,
 1.8835011943171485,
 1.6901232276156624,
 1.6734372375678963,
 1.2899846569627016,
 1.6515591043747437,
 1.4196442260395687,
 1.5895134785909621,
 1.119034630310544,
 1.0247605679022878,
 1.5772781419999897,
 1.3036652531129103,
 1.0100728759456814,
 1.045416114571532,
 1.8044512144809413,
 1.9916681367599085,
 1.1097541690006607,
 1.9451161223940594,
 1.5456209220896013,
 1.703298295990468,
 1.499621609881122,
 1.0923111767681006,
 1.1400308006481485,
 1.6355727288664916,
 1.4002326526025612,
 1.0078153869643007,
 1.9701410819761709,
 1.0132088131317647,
 1.6732307092250316,
 1.6569605457055354,
 1.0795268044498907,
 1.594874261204069,
 1.19285690856189,
 1.9096543170506945,
 1.3750418220872795,
 1.6041316772327077,
 1.606692036144942,
 1.2689280252304378,
 1.8701456582622806,
 1.925082462230103,
 1.9764466227876403,
 1.4237628707792382,
 1.1896370783892842,
 1.0659152651457626,
 1.6069051594757544,
 1.213923643323088,
 1

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.0210090519414365,
 1.9326103690258387,
 1.1083122471866134,
 1.866365801388267,
 1.5623946581128239,
 1.2304365909365833,
 1.2305939990996178,
 1.4277816350735657,
 1.695717550873457,
 1.0423835230339757,
 1.363474341179884,
 1.6664504618361209,
 1.5932890999748706,
 1.518261720195059,
 1.3417851731316044,
 1.6181475021932905,
 1.289597576501881,
 1.5151984995099117,
 1.5440567402131617,
 1.940043019464449,
 1.032202501813298,
 1.510990374346135,
 1.3365934165588063,
 1.7498341533940565,
 1.158657159458893,
 1.7045175493097418,
 1.1280964449201818,
 1.6117085891314238,
 1.6104262523654302,
 1.2898474735994794,
 1.8515422029007764,
 1.1009431789921282,
 1.5075366069201022,
 1.506368131788884,
 1.1752857224532711,
 1.9818426266040854,
 1.8956327686733423,
 1.4226742839702802,
 1.8272158407891528,
 1.2889878704801023,
 1.6135029105387422,
 1.8298250872434965,
 1.0756719708043523,
 1.2085202358962879,
 1.8508634205119416,
 1.4957280723888018,
 1.6840855584386185,
 1.0086911366229883,
 1

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 266 ms, sys: 16.2 ms, total: 282 ms
Wall time: 2.46 s
